安裝套件

In [ ]:
%pip install -q aiohttp aiofiles pandas python-dotenv openpyxl
%pip install -q miservice --no-build-isolation

建立環境變數

In [1]:
from dotenv import load_dotenv
import os

# 載入環境變數
load_dotenv()

# 讀取變數
mi_user = os.getenv('MI_USER')
mi_pass = os.getenv('MI_PASS')
mi_did = os.getenv('MI_DID')
openai_api_key = os.getenv('OPENAI_API_KEY')

查詢米家設備清單

In [2]:
!micli.py list

[
  {
    "name": "客廳監視器",
    "model": "chuangmi.camera.069a01",
    "did": "1073582979",
    "token": "6574475670786245436e5a61797a7770"
  },
  {
    "name": "車庫監視器",
    "model": "chuangmi.camera.069a01",
    "did": "1073752199",
    "token": "757173436f57515a4a6b66347a394e4e"
  },
  {
    "name": "房間監視器",
    "model": "chuangmi.camera.069a01",
    "did": "1073775259",
    "token": "37307938327a456d71706b32564e7a6b"
  },
  {
    "name": "房間監視器",
    "model": "chuangmi.camera.061a01",
    "did": "1104575738",
    "token": "4a636767467335397247434a65586163"
  },
  {
    "name": "新莊客廳監視器",
    "model": "chuangmi.camera.061a01",
    "did": "1104576170",
    "token": "737a4f37596e6e4633343771684a3764"
  },
  {
    "name": "客廳監視器",
    "model": "chuangmi.camera.ipc009",
    "did": "253578630",
    "token": "6a444371457065716657566f586c6b45"
  },
  {
    "name": "地下室監視器",
    "model": "chuangmi.camera.v2",
    "did": "66306142",
    "token": "646f416e5646624f4b345a384a763170"
  },
  {
    

篩選小愛音箱 pro

In [3]:
!micli.py list | sed 's/\x1b\[[0-9;]*m//g' | jq '.[] | select(.model == "xiaomi.wifispeaker.lx06")'

{
  "name": "小愛-地下室",
  "model": "xiaomi.wifispeaker.lx06",
  "did": "319695299",
  "token": "7044516455505337514461596b645037"
}
{
  "name": "小愛-客廳",
  "model": "xiaomi.wifispeaker.lx06",
  "did": "319865370",
  "token": "6a5131545736327550786f6e43657257"
}
{
  "name": "小愛-浴室",
  "model": "xiaomi.wifispeaker.lx06",
  "did": "403073868",
  "token": "6d424f7a66794673637a77494a5a3142"
}
{
  "name": "小愛-工作室",
  "model": "xiaomi.wifispeaker.lx06",
  "did": "604128284",
  "token": "51347a32724e7762764f6a694c6b7333"
}
{
  "name": "小愛-新莊客廳",
  "model": "xiaomi.wifispeaker.lx06",
  "did": "726638831",
  "token": "424b5834516871455758324d63745371"
}
{
  "name": "小愛-車庫",
  "model": "xiaomi.wifispeaker.lx06",
  "did": "759655877",
  "token": "593057414b4a6a58735a30514a4e4152"
}


輸出全部設備到 Excel 文件

In [4]:
import json
import pandas as pd
import subprocess

# 定義輸出文件的名稱
_OUTPUT = "device_data.xlsx"
# 預設輸出引擎
_ENGINE = "openpyxl"
# 預設分頁名稱
_DEVICE = "Device Data"


# 1. 運行 micli.py list 的函數，傳出 JSON
def run_micli_list():
    try:
        # 調用 micli.py list 並輸出設備查詢結果
        result = subprocess.run(
            # 假設 micli.py 與本代碼在同一目錄
            ["micli.py", "list"],
            capture_output=True,
            text=True
        )

        # 確認是否成功執行命令
        if result.returncode != 0:
            print(f"micli.py 執行失敗: {result.stderr}")
            return []

        # 解析 micli.py 返回的 JSON 數據
        devices = json.loads(result.stdout)
        return devices

    except Exception as e:
        print(f"運行 micli.py list 時出錯: {e}")
        return []

# 2. 將設備數據寫入 Excel 的函數
def write_to_excel(devices, output_file=_OUTPUT):
    if not devices:
        print("無法掃描到任何設備。")
        return

    # 預處理設備資料，處理缺失的字段
    processed_devices = []
    for device in devices:
        device["model"] = device.get("model", "Unknown Model")
        device["name"] = device.get("name", "Unknown Name")
        device["type"] = device.get("type", "Unknown Type")
        processed_devices.append(device)

    # 將設備資料轉換為 DataFrame
    df = pd.DataFrame(processed_devices)

    # 主資料表：包含完整設備資料
    with pd.ExcelWriter(output_file, engine=_ENGINE) as writer:
        # 寫入主資料表
        df.to_excel(writer, index=False, sheet_name=_DEVICE)

    print(f"資料已成功寫入 {output_file}")

# 3. 主程序
def main():
    print("正在掃描設備...")
    devices = run_micli_list()
    if devices:
        print(f"掃描到 {len(devices)} 個設備。")
        write_to_excel(devices)
    else:
        print("未掃描到任何設備，請檢查帳戶或網絡設置。")

if __name__ == "__main__":
    main()


正在掃描設備...
掃描到 144 個設備。
資料已成功寫入 device_data.xlsx


使用內建函數

In [5]:
from miservice import MiAccount, MiNAService
import asyncio
from aiohttp import ClientSession
import os
# 解決嵌套事件循環問題
import nest_asyncio

# 輸出文件
_OUTPUT = "device_data_detail.xlsx"
# 預設輸出引擎
_ENGINE = "openpyxl"
# 預設分頁名稱
_DEVICE = "Device Data"

# 解決嵌套事件循環問題
nest_asyncio.apply()

# 1. 掃描設備的函數
async def scan_devices():
    try:
        env_get = os.environ.get
        store = os.path.join(os.path.expanduser("~"), ".mi.token")
        async with ClientSession() as session:
            account = MiAccount(
                session,
                env_get("MI_USER"), 
                env_get("MI_PASS"), 
                store
            )
            service = MiNAService(account)
            # 使用內建的函數取得設備清單
            devices = await service.device_list()
            return devices
    except Exception as e:
        print(f"設備掃描失敗: {e}")
        return []

# 2. 寫入 Excel 的函數
def write_to_excel(devices, output_file=_OUTPUT):
    if not devices:
        print("無法掃描到任何設備。")
        return

    # 預處理設備資料，處理缺失的 model 和 name 欄位
    processed_devices = []
    for device in devices:
        device["model"] = device.get("model", "Unknown Model")
        device["name"] = device.get("name", "Unknown Name")
        processed_devices.append(device)

    # 將設備資料轉換為 DataFrame
    df = pd.DataFrame(processed_devices)

    # 主資料表：包含完整設備資料
    with pd.ExcelWriter(output_file, engine=_ENGINE) as writer:
        # 寫入主資料表
        df.to_excel(
            writer,
            index=False,
            sheet_name=_DEVICE
        )
    print(f"資料已成功寫入 {output_file}")

# 3. 主程序
async def main():
    print("正在掃描設備...")
    devices = await scan_devices()
    if devices:
        print(f"掃描到 {len(devices)} 個設備。")
        write_to_excel(devices)
    else:
        print("未掃描到任何設備，請檢查賬戶或網絡設置。")

if __name__ == "__main__":
    # 確保環境變數已設置
    if not os.getenv("MI_USER") or not os.getenv("MI_PASS"):
        print("請先設置環境變數 MI_USER 和 MI_PASS，然後重新運行腳本。")
    else:
        # 在 Jupyter Notebook 或其他事件循環環境中運行
        asyncio.run(main())

正在掃描設備...
掃描到 10 個設備。
資料已成功寫入 device_data_detail.xlsx


基本說話

In [14]:
!micli.py 5-1 您好

0
